In [1]:
# import numpy as np
import pandas as pd
import config # a python file that contains path to TIMSS data files
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ml_data_1 = pd.read_csv(config.ml_data_1, index_col=False)

In [3]:
x = ml_data_1.drop(columns='mean_PV')
x.sample()

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I
3565,No,None or very few (0–10 books),Agree a lot,Disagree a little,At least once a week,Agree a lot,Agree a lot,Some lessons,Agree a little,Agree a lot


In [4]:
y = ml_data_1.mean_PV
y.sample()

3525    427.55939
Name: mean_PV, dtype: float64

In [5]:
from kmodes.kmodes import KModes

# Instantiate the k-modes model
km = KModes(n_clusters=5, init='Huang', n_init=4, verbose=2, n_jobs=2)

# Fit the model to your categorical data
clusters = km.fit_predict(x)


Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initializing clusters
Starting iterations...
Starting iterations...
Run 1, iteration: 1/100, moves: 723, cost: 30901.0
Init: initializing centroids
Init: initializing clusters
Run 2, iteration: 1/100, moves: 1956, cost: 31445.0
Starting iterations...
Run 2, iteration: 2/100, moves: 612, cost: 31445.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1974, cost: 31350.0
Run 3, iteration: 2/100, moves: 31, cost: 31350.0
Run 4, iteration: 1/100, moves: 3106, cost: 30687.0
Run 4, iteration: 2/100, moves: 584, cost: 30687.0
Best run was number 4


In [6]:
clusters.max()

4

In [7]:
clusters

array([0, 1, 1, ..., 1, 4, 0], dtype=uint16)

Now that we have our clusters, we will add their numbers to each data point in our labels data in order to separate each cluster alone and build a separate model for each:

In [8]:
x['clusters'] = clusters
x['labels'] = y

In [9]:
x.describe()

,clusters,labels
count,8458.000000,8458.000000
mean,1.499882,385.328560
std,1.515460,62.364477
min,0.000000,194.209476
25%,0.000000,341.444794
50%,1.000000,379.916982
75%,3.000000,423.390834
max,4.000000,649.182768


In [62]:
x.sample(5)

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I,clusters,labels
5792,No,None or very few (0–10 books),Agree a lot,Agree a little,Once or twice a month,Agree a lot,Agree a lot,About half the lessons,Agree a little,Agree a lot,3,349.464136
788,No,Enough to fill three or more bookcases (more t...,Agree a little,Disagree a little,Never,Disagree a little,Disagree a lot,Every or almost every lesson,Agree a lot,Agree a lot,0,394.031688
7191,Yes,Enough to fill one shelf (11–25 books),Agree a little,Agree a little,At least once a week,Agree a little,Agree a little,Every or almost every lesson,Agree a little,Agree a lot,4,432.282972
7868,No,Enough to fill one shelf (11–25 books),Agree a lot,Agree a little,Never,Agree a little,Agree a lot,About half the lessons,Agree a little,Agree a little,1,336.710354
452,No,None or very few (0–10 books),Disagree a little,Agree a little,Once or twice a month,Disagree a little,Agree a little,Every or almost every lesson,Agree a little,Disagree a little,1,503.218330


Let's see what clusters do we have:

In [10]:
list_of_clusters = x['clusters'].unique()
list_of_clusters

array([0, 1, 2, 4, 3], dtype=uint16)

In [11]:
x[['labels','clusters']]

,labels,clusters
0,410.811348,0
1,433.181462,1
2,418.027694,1
3,467.300384,0
4,362.964464,0
...,...,...
8453,401.279696,1
8454,438.474212,4
8455,401.756130,1
8456,312.155188,4


Let's put each cluster in a different dataframe:

In [12]:
y_0 = x[['labels','clusters']].query('clusters==0')
x_0 = x.query('clusters==0')
y_1 = x[['labels','clusters']].query('clusters==1')
x_1 = x.query('clusters==1')
y_2 = x[['labels','clusters']].query('clusters==2')
x_2 = x.query('clusters==2')
y_3 = x[['labels','clusters']].query('clusters==3')
x_3 = x.query('clusters==3')
y_4 = x[['labels','clusters']].query('clusters==4')
x_4 = x.query('clusters==4')

In [18]:
x_3.count()

BSBG05D     1172
BSBG04      1172
BSBG13E     1172
BSBG13D     1172
BSBG14A     1172
BSBM16A     1172
BSBM17D     1172
BSBM18C     1172
BSBM20E     1172
BSBM20I     1172
clusters    1172
labels      1172
dtype: int64

Let's do some hyperparameters tuning of xgboost algorithm executed on "x_0" dataset using optuna:

In [19]:
import optuna
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [21]:
x_3.columns

Index(['BSBG05D', 'BSBG04', 'BSBG13E', 'BSBG13D', 'BSBG14A', 'BSBM16A',
       'BSBM17D', 'BSBM18C', 'BSBM20E', 'BSBM20I', 'clusters', 'labels'],
      dtype='object')

Let's keep only necessary features by deleting the 'clusters'  and 'labels' columns

In [22]:
df_list_features = [x_0, x_1, x_2, x_3, x_4]
df_list_labels = [y_0, y_1, y_3, y_4]
for df in df_list_features:
    del df['clusters']
    del df['labels']
for df in df_list_labels:
    del df['clusters']

In [23]:
x_3.columns

Index(['BSBG05D', 'BSBG04', 'BSBG13E', 'BSBG13D', 'BSBG14A', 'BSBM16A',
       'BSBM17D', 'BSBM18C', 'BSBM20E', 'BSBM20I'],
      dtype='object')

In [24]:
y_0.columns

Index(['labels'], dtype='object')

Let's start defining our objective function used by optuna, where we will define our parameters:

In [ ]:
#The following line is just to display, on hover, parameters of xgboost regressor

xgb.XGBRFRegressor()

In [25]:
x_0.dtypes

BSBG05D    object
BSBG04     object
BSBG13E    object
BSBG13D    object
BSBG14A    object
BSBM16A    object
BSBM17D    object
BSBM18C    object
BSBM20E    object
BSBM20I    object
dtype: object

In [26]:
x_0 = x_0.astype('category')
x_1 = x_1.astype('category')
x_2 = x_2.astype('category')
x_3 = x_3.astype('category')
x_4 = x_4.astype('category')

In [27]:
x_3.dtypes

BSBG05D    category
BSBG04     category
BSBG13E    category
BSBG13D    category
BSBG14A    category
BSBM16A    category
BSBM17D    category
BSBM18C    category
BSBM20E    category
BSBM20I    category
dtype: object

In [120]:
#import category_encoders as ce
#sklearn.set_config(transform_output="pandas")

#encoder = ce.CatBoostEncoder(cols=['BSBG05D','BSBG04','BSBG13E','BSBG13D','BSBG14A','BSBM16A','BSBM17D','BSBM18C','BSBM20E','BSBM20I'])
#encoder.fit(x_3, y_3)
#X_cleaned = encoder.transform(x_3)


In [31]:
def objective( trail, data = x_3, target = y_3):
    train_x_3, test_x_3, train_y_3, test_y_3 = train_test_split(data,target, test_size=.20, random_state=30)
    param = {
        'tree_method':'hist',
        'enable_categorical':True, # mandatory when we have categorical features
        'lambda': trail.suggest_loguniform('lambda', 1e-10, 10), # lambda uses L2 regularization
        'alpha': trail.suggest_loguniform('alpha', 1e-10, 10),
        'colsample_bytree':trail.suggest_categorical('colsample_bytree', [.1,.2,.3,.4,.5,.8,.9]),
        'subsample':trail.suggest_categorical('subsample', [.1,.2,.3,.4,.5,.6,.7,.8,.9]),
        'learning_rate':trail.suggest_categorical('learning_rate', [.00001,.0003,.02,.004,.5,1,10]),
        'n_estimator': 30000,
        'max_depth':trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11]),
        'random_state': trail.suggest_categorical('random_state', [10,30,200,5000]),
        'min_child_weight': trail.suggest_int('min_child_weight',1,200),
        'verbosity': 3
    }
    xgb_x_3_model = xgb.XGBRFRegressor(**param)
    xgb_x_3_model.fit(train_x_3,train_y_3,eval_set = [(test_x_3,test_y_3)], verbose = True)
    prediction_xgb = xgb_x_3_model.predict(test_x_3)
    rmse = mean_squared_error(test_y_3,prediction_xgb)
    return rmse
    

In [33]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective, n_trials = 250)
find_param.best_params

[I 2023-11-07 11:20:54,210] A new study created in memory with name: no-name-ad2505d5-4073-4053-837f-0d446662c03c
/tmp/ipykernel_30561/3947309842.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trail.suggest_loguniform('lambda', 1e-10, 10), # lambda uses L2 regularization
/tmp/ipykernel_30561/3947309842.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trail.suggest_loguniform('alpha', 1e-10, 10),
/home/aazzioui/Documents/WS/TIMSS-2019_student_score_prediction/env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:20:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, Us

[11:20:54] ======== Monitor (0): HostSketchContainer ========
[11:20:54] AllReduce: 5e-06s, 1 calls @ 5us

[11:20:54] MakeCuts: 1.5e-05s, 1 calls @ 15us

[11:20:54] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3
[0]	validation_0-rmse:54.50662


[I 2023-11-07 11:20:54,488] Trial 0 finished with value: 2970.9715399947077 and parameters: {'lambda': 5.6425186560814726e-05, 'alpha': 5.689259856022357e-10, 'colsample_bytree': 0.2, 'subsample': 0.4, 'learning_rate': 1, 'max_depth': 9, 'random_state': 10, 'min_child_weight': 70}. Best is trial 0 with value: 2970.9715399947077.


[11:20:54] ======== Monitor (0): Learner ========
[11:20:54] Configure: 0.000933s, 1 calls @ 933us

[11:20:54] EvalOneIter: 0.000309s, 1 calls @ 309us

[11:20:54] GetGradient: 8e-06s, 1 calls @ 8us

[11:20:54] PredictRaw: 5e-06s, 1 calls @ 5us

[11:20:54] UpdateOneIter: 0.184067s, 1 calls @ 184067us

[11:20:54] ======== Monitor (0): GBTree ========
[11:20:54] BoostNewTrees: 0.178759s, 1 calls @ 178759us

[11:20:54] CommitModel: 3e-06s, 1 calls @ 3us

[11:20:54] ======== Monitor (0): HistUpdater ========
[11:20:54] BuildHistogram: 0.025555s, 154 calls @ 25555us

[11:20:54] EvaluateSplits: 0.033634s, 254 calls @ 33634us

[11:20:54] InitData: 0.019256s, 100 calls @ 19256us

[11:20:54] InitRoot: 0.064467s, 100 calls @ 64467us

[11:20:54] LeafPartition: 1.6e-05s, 100 calls @ 16us

[11:20:54] UpdatePosition: 0.053734s, 154 calls @ 53734us

[11:20:54] UpdateTree: 0.175308s, 100 calls @ 175308us

[11:20:54] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3
[11:20:54] ======== Monit

/tmp/ipykernel_30561/3947309842.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trail.suggest_loguniform('lambda', 1e-10, 10), # lambda uses L2 regularization
/tmp/ipykernel_30561/3947309842.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trail.suggest_loguniform('alpha', 1e-10, 10),
/home/aazzioui/Documents/WS/TIMSS-2019_student_score_prediction/env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:20:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


...

/home/aazzioui/Documents/WS/TIMSS-2019_student_score_prediction/env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:22:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:53.00653


[I 2023-11-07 11:22:57,888] Trial 249 finished with value: 2809.6916843042604 and parameters: {'lambda': 2.0624041049804055e-07, 'alpha': 1.0947430948613559e-08, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 1, 'max_depth': 5, 'random_state': 5000, 'min_child_weight': 39}. Best is trial 239 with value: 2769.2226865647212.


[11:22:57] ======== Monitor (0): Learner ========
[11:22:57] Configure: 0.000897s, 1 calls @ 897us

[11:22:57] EvalOneIter: 0.000584s, 1 calls @ 584us

[11:22:57] GetGradient: 7e-06s, 1 calls @ 7us

[11:22:57] PredictRaw: 2e-06s, 1 calls @ 2us

[11:22:57] UpdateOneIter: 0.216928s, 1 calls @ 216928us

[11:22:57] ======== Monitor (0): GBTree ========
[11:22:57] BoostNewTrees: 0.215826s, 1 calls @ 215826us

[11:22:57] CommitModel: 2e-06s, 1 calls @ 2us

[11:22:57] ======== Monitor (0): HistUpdater ========
[11:22:57] BuildHistogram: 0.083617s, 400 calls @ 83617us

[11:22:57] EvaluateSplits: 0.055492s, 500 calls @ 55492us

[11:22:57] InitData: 0.00714s, 100 calls @ 7140us

[11:22:57] InitRoot: 0.003721s, 100 calls @ 3721us

[11:22:57] LeafPartition: 1.7e-05s, 100 calls @ 17us

[11:22:57] UpdatePosition: 0.056576s, 466 calls @ 56576us

[11:22:57] UpdateTree: 0.206213s, 100 calls @ 206213us

[11:22:57] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3


{'lambda': 7.387304258730289e-07,
 'alpha': 1.2496756424022223e-08,
 'colsample_bytree': 0.8,
 'subsample': 0.5,
 'learning_rate': 1,
 'max_depth': 10,
 'random_state': 5000,
 'min_child_weight': 37}

In [37]:
find_param.best_value

2769.2226865647212

In [35]:
find_param.best_params

{'lambda': 7.387304258730289e-07,
 'alpha': 1.2496756424022223e-08,
 'colsample_bytree': 0.8,
 'subsample': 0.5,
 'learning_rate': 1,
 'max_depth': 10,
 'random_state': 5000,
 'min_child_weight': 37}

In [50]:
# put trials details in a dataframe:
df_trials = find_param.trials_dataframe()
df_trials

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,2970.971540,2023-11-07 11:20:54.213326,2023-11-07 11:20:54.488145,0 days 00:00:00.274819,5.689260e-10,0.2,5.642519e-05,1.000,9,70,10,0.4,COMPLETE
1,1,3033.374183,2023-11-07 11:20:54.490522,2023-11-07 11:20:54.873584,0 days 00:00:00.383062,3.670110e-08,0.9,7.343380e-07,0.020,5,47,200,0.8,COMPLETE
2,2,3044.021090,2023-11-07 11:20:54.876292,2023-11-07 11:20:55.072830,0 days 00:00:00.196538,1.132035e-06,0.3,2.295534e-02,0.004,10,63,5000,0.8,COMPLETE
3,3,3043.630121,2023-11-07 11:20:55.075379,2023-11-07 11:20:55.352063,0 days 00:00:00.276684,1.950130e-07,0.5,3.614264e-04,0.004,11,42,200,0.5,COMPLETE
4,4,2837.375833,2023-11-07 11:20:55.353347,2023-11-07 11:20:55.700594,0 days 00:00:00.347247,5.047260e-05,0.8,3.044737e-10,0.500,6,41,30,0.7,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,2810.773970,2023-11-07 11:22:55.706075,2023-11-07 11:22:56.276291,0 days 00:00:00.570216,4.254445e-08,0.8,5.348424e-07,1.000,7,33,5000,0.7,COMPLETE
246,246,2862.753106,2023-11-07 11:22:56.281856,2023-11-07 11:22:56.570657,0 days 00:00:00.288801,5.056849e-09,0.3,3.332647e-08,1.000,5,36,5000,0.5,COMPLETE
247,247,2793.540443,2023-11-07 11:22:56.582441,2023-11-07 11:22:56.899061,0 days 00:00:00.316620,8.530955e-08,0.8,2.000098e-06,1.000,5,27,5000,0.5,COMPLETE
248,248,2782.634345,2023-11-07 11:22:56.900458,2023-11-07 11:22:57.471331,0 days 00:00:00.570873,1.395296e-08,0.8,1.131770e-07,1.000,5,31,5000,0.5,COMPLETE


In [52]:
find_param.best_trial

FrozenTrial(number=239, state=1, values=[2769.2226865647212], datetime_start=datetime.datetime(2023, 11, 7, 11, 22, 53, 269892), datetime_complete=datetime.datetime(2023, 11, 7, 11, 22, 53, 772251), params={'lambda': 7.387304258730289e-07, 'alpha': 1.2496756424022223e-08, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 1, 'max_depth': 10, 'random_state': 5000, 'min_child_weight': 37}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lambda': FloatDistribution(high=10.0, log=True, low=1e-10, step=None), 'alpha': FloatDistribution(high=10.0, log=True, low=1e-10, step=None), 'colsample_bytree': CategoricalDistribution(choices=(0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 0.9)), 'subsample': CategoricalDistribution(choices=(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)), 'learning_rate': CategoricalDistribution(choices=(1e-05, 0.0003, 0.02, 0.004, 0.5, 1, 10)), 'max_depth': CategoricalDistribution(choices=(3, 4, 5, 6, 7, 8, 9, 10, 11)), 'random_state': CategoricalDistrib

In [51]:
display(df_trials.iloc[239])

number                                            239
value                                     2769.222687
datetime_start             2023-11-07 11:22:53.269892
datetime_complete          2023-11-07 11:22:53.772251
duration                       0 days 00:00:00.502359
params_alpha                                      0.0
params_colsample_bytree                           0.8
params_lambda                                0.000001
params_learning_rate                              1.0
params_max_depth                                   10
params_min_child_weight                            37
params_random_state                              5000
params_subsample                                  0.5
state                                        COMPLETE
Name: 239, dtype: object